## Titel: Import data
## Author: Achraf Aboukinana

## Beschreibung des Codes

Dieser Code dient der Analyse, Bereinigung und Kombination von Datensätzen, um eine zentrale und bereinigte Datenbasis zu erstellen. Die Hauptaufgaben sind wie folgt:

---

## **1. Datenimport**
- Die drei Datensätze `kiwo.csv`, `umsatzdaten_gekuerzt.csv` und `wetterdaten.csv` werden eingelesen.
- Die Spalte `Datum` wird als Datumsformat (`datetime`) geparst.

---

## **2. Datenexploration**
Eine Funktion `explore_data` analysiert jeden DataFrame:
- Form des DataFrames (Zeilen und Spalten).
- Vorschau der ersten fünf Zeilen.
- Spaltennamen, Datentypen und fehlende Werte pro Spalte.
- Beschreibende Statistiken.
- Anzahl der Duplikate.
- Zeitraum der Daten (falls `Datum`-Spalte vorhanden).

Diese Funktion wird auf alle Datensätze angewendet.

---

## **3. Zusammenführung der Datensätze**
- **Erster Merge**: `umsatzdaten_gekuerzt_data` und `wetter_data` werden anhand der Spalte `Datum` zusammengeführt.
- **Zweiter Merge**: Die zusammengeführten Daten werden mit `kiwo_data` kombiniert, wodurch eine neue Spalte `KielerWoche` entsteht:
  - Fehlende Werte in `KielerWoche` (für nicht-Kieler-Woche-Tage) werden mit `0` gefüllt.

---

## **4. Analyse fehlender Werte**
- Anzahl der fehlenden Werte (`NaN`) pro Spalte wird berechnet.
- Zeilen mit fehlenden Werten werden entfernt.

---

## **5. Speichern der Ergebnisse**
- Der kombinierte Datensatz wird in zwei Versionen gespeichert:
  - **Mit NaN-Werten**: `final_data_withNaN.csv`.
  - **Ohne NaN-Werte**: `final_data.csv`.

---

## **6. Vergleich der Zeilenanzahl**
- Die Anzahl der Zeilen vor und nach der Bereinigung wird verglichen, um den Verlust durch das Entfernen von `NaN`-Werten zu zeigen.

---

## **Zweck des Codes**
1. **Datenbereinigung**: Sicherstellung von konsistenten und vollständigen Daten.
2. **Kombination der Datensätze**: Erstellung einer zentralen, einheitlichen Datenbasis.
3. **Markierung besonderer Tage**: Identifikation der Kieler Woche-Tage (`KielerWoche`).
4. **Analyse-Ready**: Der bereinigte Datensatz dient als Grundlage für Analysen oder Modelle.


In [74]:


# Funktion zur Datenexploration
def explore_data(df, name):
    print(f"\n{'='*50}\nExploration für {name}\n{'='*50}")
    
    # Überblick
    print(f"Form: {df.shape[0]} Zeilen, {df.shape[1]} Spalten")
    print("Erste 5 Zeilen:")
    print(df.head())
    
    # Spaltennamen und Typen
    print("\nSpalten und Datentypen:")
    print(df.dtypes)
    
    # Fehlende Werte
    print("\nFehlende Werte pro Spalte:")
    print(df.isna().sum())
    
    # Beschreibende Statistik
    print("\nBeschreibende Statistik:")
    print(df.describe(include='all'))
    
    # Duplikate
    duplicate_count = df.duplicated().sum()
    print(f"\nAnzahl der Duplikate: {duplicate_count}")

    # Wichtige Spalten
    if 'Datum' in df.columns:
        print("\nZeitraum der Daten:")
        print(f"Min Datum: {df['Datum'].min()}, Max Datum: {df['Datum'].max()}")
    else:
        print("\nKeine 'Datum'-Spalte in diesem DataFrame.")


In [75]:
import pandas as pd
import holidays

data_pfad= 'kaggel/train.csv'
model_data_pfad = 'final_data_train.csv'
# Import Data
kiwo_data = pd.read_csv('kaggel/kiwo.csv', parse_dates=['Datum'])
umsatzdaten_gekuerzt_data = pd.read_csv(data_pfad, parse_dates=['Datum'])
wetter_data = pd.read_csv('kaggel/wetter.csv', parse_dates=['Datum'])


In [76]:

# Exploration der einzelnen Dateien
explore_data(kiwo_data, "DATA/Kiwo Data")
explore_data(umsatzdaten_gekuerzt_data, "DATA/Umsatzdaten Gekürzt")
explore_data(wetter_data, "DATA/Wetter Data")



Exploration für DATA/Kiwo Data
Form: 72 Zeilen, 2 Spalten
Erste 5 Zeilen:
       Datum  KielerWoche
0 2012-06-16            1
1 2012-06-17            1
2 2012-06-18            1
3 2012-06-19            1
4 2012-06-20            1

Spalten und Datentypen:
Datum          datetime64[ns]
KielerWoche             int64
dtype: object

Fehlende Werte pro Spalte:
Datum          0
KielerWoche    0
dtype: int64

Beschreibende Statistik:
                     Datum  KielerWoche
count                   72         72.0
mean   2015-12-23 00:00:00          1.0
min    2012-06-16 00:00:00          1.0
25%    2014-03-24 00:00:00          1.0
50%    2015-12-23 00:00:00          1.0
75%    2017-09-22 00:00:00          1.0
max    2019-06-30 00:00:00          1.0
std                    NaN          0.0

Anzahl der Duplikate: 0

Zeitraum der Daten:
Min Datum: 2012-06-16 00:00:00, Max Datum: 2019-06-30 00:00:00

Exploration für DATA/Umsatzdaten Gekürzt
Form: 9334 Zeilen, 4 Spalten
Erste 5 Zeilen:
        id   

In [77]:
import pandas as pd
import holidays

# Merge Umsatzdaten mit Wetterdaten auf 'Datum'
merged_data = pd.merge(umsatzdaten_gekuerzt_data, wetter_data, on='Datum', how='inner')

# Füge KielerWoche-Information hinzu und ersetze fehlende Werte durch 0
final_data = pd.merge(merged_data, kiwo_data, on='Datum', how='left')
final_data['KielerWoche'] = final_data['KielerWoche'].fillna(0).astype(int)


In [78]:

# Anzahl der NaN-Werte überprüfen
nan_counts = final_data.isna().sum()
print("Anzahl der NaN-Werte pro Spalte:")
print(nan_counts)

# Entferne Zeilen mit NaN-Werten
clean_data = final_data.dropna()


Anzahl der NaN-Werte pro Spalte:
id                        0
Datum                     0
Warengruppe               0
Umsatz                    0
Bewoelkung               54
Temperatur                0
Windgeschwindigkeit       0
Wettercode             2309
KielerWoche               0
dtype: int64


In [79]:

# **Zeitabhängige Variablen hinzufügen**
clean_data['Wochentag'] = clean_data['Datum'].dt.dayofweek  # Montag=0, Sonntag=6
clean_data['Monat'] = clean_data['Datum'].dt.month
clean_data['Jahr'] = clean_data['Datum'].dt.year

clean_data['Jahreszeit'] = clean_data['Datum'].dt.month % 12 // 3 + 1  # Frühling=1, Sommer=2, Herbst=3, Winter=4


C:\Users\lorda\AppData\Local\Temp\ipykernel_20564\1988144829.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['Wochentag'] = clean_data['Datum'].dt.dayofweek  # Montag=0, Sonntag=6
C:\Users\lorda\AppData\Local\Temp\ipykernel_20564\1988144829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['Monat'] = clean_data['Datum'].dt.month
C:\Users\lorda\AppData\Local\Temp\ipykernel_20564\1988144829.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [80]:



# Jahreszeit bestimmen
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Frühling"
    elif month in [6, 7, 8]:
        return "Sommer"
    elif month in [9, 10, 11]:
        return "Herbst"

clean_data['Jahreszeit'] = clean_data['Monat'].apply(get_season)

# Gefühl abhängig von Temperatur und Jahreszeit
def temperature_feeling(temp, season):
    if season == "Winter":
        if temp <= 0:
            return "Kalt"
        elif temp <= 10:
            return "Mild"
        else:
            return "Warm"
    elif season == "Frühling":
        if temp <= 10:
            return "Kalt"
        elif temp <= 20:
            return "Mild"
        else:
            return "Warm"
    elif season == "Sommer":
        if temp <= 15:
            return "Kalt"
        elif temp <= 25:
            return "Mild"
        else:
            return "Warm"
    elif season == "Herbst":
        if temp <= 10:
            return "Kalt"
        elif temp <= 20:
            return "Mild"
        else:
            return "Warm"

clean_data['Gefühl'] = clean_data.apply(lambda x: temperature_feeling(x['Temperatur'], x['Jahreszeit']), axis=1)

# Dummy-Encoding
gefühl_dummies = pd.get_dummies(clean_data['Gefühl'],prefix='gefühl')

# Dummy-Variablen anhängen und Originalspalten löschen
clean_data = pd.concat([clean_data,gefühl_dummies], axis=1)
#gefühl_dummies.drop(columns=['Gefühl'], inplace=True)

clean_data.to_csv('final_data_x.csv',index=False)

C:\Users\lorda\AppData\Local\Temp\ipykernel_20564\171020470.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['Jahreszeit'] = clean_data['Monat'].apply(get_season)
C:\Users\lorda\AppData\Local\Temp\ipykernel_20564\171020470.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['Gefühl'] = clean_data.apply(lambda x: temperature_feeling(x['Temperatur'], x['Jahreszeit']), axis=1)


In [81]:
# **Feiertage hinzufügen**
def get_feiertage(year):
    de_holidays = holidays.Germany(years=year)
    return list(de_holidays.keys())

# Feiertagsinformationen für alle Jahre im Datensatz
feiertage_set = {holiday for year in clean_data['Datum'].dt.year.unique() for holiday in get_feiertage(year)}

# Feiertagsspalte erstellen
clean_data['Feiertag'] = clean_data['Datum'].apply(lambda x: 1 if x.date() in feiertage_set else 0)

In [82]:



# **Besondere Events (KielLauf, Kieler Triathlon) hinzufügen**
kieler_triathlon_daten = pd.to_datetime(['2013-08-04', '2014-08-03', '2015-08-02', '2016-08-07', '2017-08-06', 
                                         '2018-08-05', '2019-08-04'])
kiellauf_daten = pd.to_datetime(["2013-09-08", "2014-09-14", "2015-09-13", "2016-09-11", 
                                 "2017-09-10", "2018-09-09", "2019-09-08"])
important_games_germany = pd.to_datetime([
    # FIFA World Cup 2014
    "2014-07-08",  # Germany 7–1 Brazil (Semifinal)
    "2014-07-13",  # Germany 1–0 Argentina (Final)

    # UEFA Euro 2016
    "2016-07-02",  # Germany 1–1 Italy (6–5 on penalties, Quarterfinal)
    "2016-07-07",  # Germany 0–2 France (Semifinal)

    # FIFA Confederations Cup 2017
    "2017-06-19",  # Germany 3–2 Australia
    "2017-06-22",  # Germany 1–1 Chile
    "2017-06-25",  # Germany 3–1 Cameroon
    "2017-06-29",  # Germany 4–1 Mexico (Semifinal)
    "2017-07-02",  # Germany 1–0 Chile (Final)

    # FIFA World Cup 2018
    "2018-06-17",  # Germany 0–1 Mexico (Group stage)
    "2018-06-27",  # Germany 0–2 South Korea (Group stage, eliminated)

    # Olympic Games 2016 (Men's Football)
    "2016-08-20"   # Germany 1–1 Brazil (4–5 on penalties, Final)
])

clean_data['KielLauf'] = clean_data['Datum'].apply(lambda x: 1 if x.date() in kiellauf_daten.date else 0)
clean_data['Kieler_Triathlon'] = clean_data['Datum'].apply(lambda x: 1 if x.date() in kieler_triathlon_daten.date else 0)
clean_data['Fußball'] = clean_data['Datum'].apply(lambda x: 1 if x.date() in important_games_germany.date else 0)


In [83]:

# **PaycheckEffect (Gehaltszahlungen)**
clean_data['PaycheckEffect'] = clean_data['Datum'].apply(
    lambda x: 1 if x.day >= 27 or x.day <= 5 else 0
)

# Inflation sensitivity mapping based on WarenGruppe
sensitivity_mapping = {
    1: "High",   # Brot
    2: "High",   # Brötchen
    3: "Moderate",  # Croissant
    4: "Moderate",  # Konditorei
    5: "Moderate",  # Kuchen
    6: "High"    # Saisonbrot
}

# Map the sensitivity to the WarenGruppe
clean_data["InflationSensitivity"] = clean_data["Warengruppe"].map(sensitivity_mapping)

# Perform dummy encoding for sensitivity levels, excluding 'Low'
dummy_encoded = pd.get_dummies(clean_data["InflationSensitivity"], prefix="Sensitivity").drop(columns=["Sensitivity_Low"], errors="ignore")

# Concatenate the dummy columns with the original DataFrame
clean_data = pd.concat([clean_data, dummy_encoded], axis=1)


In [84]:


# **Windkategorie basierend auf Windgeschwindigkeit**
def wind_kategorie(wind):
    if wind < 10:
        return "Nicht windig"
    elif wind <= 20:
        return "Windig"
    else:
        return "Sehr windig" #TODO zu => sehr

clean_data['Windkategorie'] = clean_data['Windgeschwindigkeit'].apply(wind_kategorie)

# Dummy-Variablen für Windkategorien erstellen
wind_dummies = pd.get_dummies(clean_data['Windkategorie'], prefix="Wind")
clean_data = pd.concat([clean_data, wind_dummies], axis=1)




In [85]:

# **Dummy-Encoding für Jahreszeit und Wetterkategorie**
jahreszeit_dummies = pd.get_dummies(clean_data['Jahreszeit'], prefix="Jahreszeit")

# Dummy-Variablen anhängen und Originalspalten löschen
clean_data = pd.concat([clean_data, jahreszeit_dummies], axis=1)


In [86]:
from datetime import datetime, timedelta
import pandas as pd

ferien_kiel = [
    # Schulferien 2013
    ('2013-01-01', '2013-01-05'),  # Weihnachtsferien
    ('2013-03-25', '2013-04-06'),  # Osterferien
    ('2013-06-24', '2013-08-03'),  # Sommerferien
    ('2013-10-04', '2013-10-19'),  # Herbstferien
    ('2013-12-23', '2014-01-04'),  # Weihnachtsferien

    # Schulferien 2014
    ('2014-04-07', '2014-04-22'),  # Osterferien
    ('2014-07-14', '2014-08-23'),  # Sommerferien
    ('2014-10-20', '2014-11-01'),  # Herbstferien
    ('2014-12-22', '2015-01-06'),  # Weihnachtsferien

    # Schulferien 2015
    ('2015-04-01', '2015-04-18'),  # Osterferien
    ('2015-07-16', '2015-08-29'),  # Sommerferien
    ('2015-10-19', '2015-10-31'),  # Herbstferien
    ('2015-12-23', '2016-01-06'),  # Weihnachtsferien

    # Schulferien 2016
    ('2016-03-24', '2016-04-09'),  # Osterferien
    ('2016-07-25', '2016-09-03'),  # Sommerferien
    ('2016-10-17', '2016-10-29'),  # Herbstferien
    ('2016-12-23', '2017-01-07'),  # Weihnachtsferien

    # Schulferien 2017
    ('2017-04-07', '2017-04-22'),  # Osterferien
    ('2017-07-24', '2017-09-02'),  # Sommerferien
    ('2017-10-16', '2017-10-28'),  # Herbstferien
    ('2017-12-21', '2018-01-06'),  # Weihnachtsferien

    # Schulferien 2018
    ('2018-03-29', '2018-04-13'),  # Osterferien
    ('2018-07-09', '2018-08-18'),  # Sommerferien
    ('2018-10-01', '2018-10-13'),  # Herbstferien
    ('2018-12-21', '2019-01-04'),  # Weihnachtsferien

    # Schulferien 2019
    ('2019-04-04', '2019-04-18'),  # Osterferien
    ('2019-07-01', '2019-08-10'),  # Sommerferien
    ('2019-10-04', '2019-10-19'),  # Herbstferien
    ('2019-12-23', '2020-01-06'),  # Weihnachtsferien

    # Schulferien 2020
    ('2020-03-30', '2020-04-17'),  # Osterferien
    ('2020-06-29', '2020-08-08'),  # Sommerferien
    ('2020-10-05', '2020-10-17'),  # Herbstferien
    ('2020-12-23', '2021-01-06'),  # Weihnachtsferien
]

# Konvertiere Ferien zu einer Liste von Datumsobjekten
ferien_tage = []
for start, end in ferien_kiel:
    start_date = datetime.strptime(start, '%Y-%m-%d')
    end_date = datetime.strptime(end, '%Y-%m-%d')
    while start_date <= end_date:
        ferien_tage.append(start_date)
        start_date += timedelta(days=1)


# Set der Ferientage für schnellen Zugriff
ferien_set = set(ferien_tage)

# Spalte hinzufügen: 1 für Ferien, 0 für keine Ferien
clean_data['Ferien'] = clean_data['Datum'].apply(lambda x: 1 if x in ferien_set else 0)




In [87]:


# Neue Spalte basierend auf Wochentagen erstellen
clean_data['Tag_Kategorie'] = clean_data['Wochentag'].apply(lambda x: 'Montag_bis_Freitag' if x < 5 else 'Wochenende')
# Neue Spalte für Wochenenden erstellen
clean_data['Wochenende'] = clean_data['Wochentag'].apply(lambda x: 1 if x >= 5 else 0)




In [88]:
inflation_data= pd.read_csv('DATA/inflation.csv')
print(inflation_data.columns)
subset_inflation = inflation_data.drop(columns=[ 'Verbraucherpreisindex', 'Veränderung_Vorjahresmonat',
       'Veränderung_Vormonat',
       'Inflation_Kategorisierung_vormonat_Neutral',
       'Inflation_Kategorisierung_vormonat_Positiv',
       'Inflation_Kategorisierung_vormonat_Neutral.1',
       'Inflation_Kategorisierung_vormonat_Positiv.1'])
print(subset_inflation.columns)

Index(['Jahr', 'Monat', 'Verbraucherpreisindex', 'Veränderung_Vorjahresmonat',
       'Veränderung_Vormonat', 'Inflation_Kategorisierung',
       'Inflation_Kategorisierung_Neutral',
       'Inflation_Kategorisierung_Positiv',
       'Inflation_Kategorisierung_vormonat_Neutral',
       'Inflation_Kategorisierung_vormonat_Positiv',
       'Inflation_Kategorisierung_vormonat_Neutral.1',
       'Inflation_Kategorisierung_vormonat_Positiv.1'],
      dtype='object')
Index(['Jahr', 'Monat', 'Inflation_Kategorisierung',
       'Inflation_Kategorisierung_Neutral',
       'Inflation_Kategorisierung_Positiv'],
      dtype='object')


In [89]:


clean_data = pd.merge(clean_data, subset_inflation, on=['Jahr', 'Monat'], how='left')
print(clean_data.columns)


Index(['id', 'Datum', 'Warengruppe', 'Umsatz', 'Bewoelkung', 'Temperatur',
       'Windgeschwindigkeit', 'Wettercode', 'KielerWoche', 'Wochentag',
       'Monat', 'Jahr', 'Jahreszeit', 'Gefühl', 'gefühl_Kalt', 'gefühl_Mild',
       'gefühl_Warm', 'Feiertag', 'KielLauf', 'Kieler_Triathlon', 'Fußball',
       'PaycheckEffect', 'InflationSensitivity', 'Sensitivity_High',
       'Sensitivity_Moderate', 'Windkategorie', 'Wind_Nicht windig',
       'Wind_Sehr windig', 'Wind_Windig', 'Jahreszeit_Frühling',
       'Jahreszeit_Herbst', 'Jahreszeit_Sommer', 'Jahreszeit_Winter', 'Ferien',
       'Tag_Kategorie', 'Wochenende', 'Inflation_Kategorisierung',
       'Inflation_Kategorisierung_Neutral',
       'Inflation_Kategorisierung_Positiv'],
      dtype='object')


In [90]:

# **Dummy-Encoding für Jahreszeit und Wetterkategorie**
wochentag_dummy = pd.get_dummies(clean_data['Wochentag'], prefix="Wochentag")
warrengruppe_dummy = pd.get_dummies(clean_data['Warengruppe'], prefix="Warengruppe")


# Dummy-Variablen anhängen und Originalspalten löschen
clean_data = pd.concat([clean_data, wochentag_dummy,warrengruppe_dummy], axis=1)


In [91]:



# Spalte umbenennen
clean_data.rename(columns={"Wochentag_0": "Montag"}, inplace=True)
clean_data.rename(columns={"Wochentag_1": "Dienstag"}, inplace=True)
clean_data.rename(columns={"Wochentag_2": "Mittwoch"}, inplace=True)
clean_data.rename(columns={"Wochentag_3": "Donnerstag"}, inplace=True)
clean_data.rename(columns={"Wochentag_4": "Freitag"}, inplace=True)
clean_data.rename(columns={"Wochentag_5": "Samstag"}, inplace=True)
clean_data.rename(columns={"Wochentag_6": "Sonntag"}, inplace=True)


# Spalte umbenennen
clean_data.rename(columns={"Warengruppe_1": "Brot"}, inplace=True)
clean_data.rename(columns={"Warengruppe_2": "Brötchen"}, inplace=True)
clean_data.rename(columns={"Warengruppe_3": "Croissant"}, inplace=True)
clean_data.rename(columns={"Warengruppe_4": "Konditorei"}, inplace=True)
clean_data.rename(columns={"Warengruppe_5": "Kuchen"}, inplace=True)
clean_data.rename(columns={"Warengruppe_6": "Saisonbrot"}, inplace=True)


In [92]:
# Identify boolean columns
boolean_columns = clean_data.select_dtypes(include=['bool']).columns

# Convert boolean columns to numeric
clean_data[boolean_columns] = clean_data[boolean_columns].astype(int)


clean_data.to_csv(model_data_pfad, index=False)